# Décodage du charabia avec linguistica
Produire trois version du texte avec :
- la phrase d'origine
- la phrase découpée
- la glose pour chaque morphème

À terme produire un document LaTeX avec en plus une ligne pour noter la traduction

In [1]:
import linguistica as lxa
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import re,tabulate
from itertools import combinations
lxa.__version__  # show version number

'5.2.1'

## Lire le corpus

In [2]:
charabia="Aveugle"
repName="/Users/gilles/ownCloud/Cours/Bordeaux/M2-TraitementsCorpus/"
fName="Corpus%s-Sentenced-Ordered.txt"%charabia
# lxaCorpus = lxa.read_corpus(repName+fName)

In [3]:
texteBrut=pd.read_csv(repName+fName,encoding="utf8",header=None,sep="\t").values.tolist()
texteBrut=[t[0] for t in texteBrut]
compW=re.compile(u"(\W)+",re.U)
texteBrut=[re.sub(compW," ",l) for l in texteBrut]
lxaCorpus = lxa.from_corpus(texteBrut)

In [4]:
%store -r conteLines

lineConte={}
nContes=[]
nConte=""
nDebutConte=0
for nLine,line in enumerate(texteBrut):
    m=re.search(u"^(\d+)",line)
    if m:
        nConte=m.group(1)
        nContes.append(nConte)
    lineConte[nLine]=nConte
lineConte

{0: u'001',
 1: u'001',
 2: u'001',
 3: u'001',
 4: u'001',
 5: u'001',
 6: u'001',
 7: u'001',
 8: u'001',
 9: u'001',
 10: u'001',
 11: u'001',
 12: u'001',
 13: u'001',
 14: u'001',
 15: u'001',
 16: u'001',
 17: u'001',
 18: u'001',
 19: u'001',
 20: u'001',
 21: u'001',
 22: u'001',
 23: u'001',
 24: u'001',
 25: u'001',
 26: u'001',
 27: u'001',
 28: u'001',
 29: u'001',
 30: u'001',
 31: u'001',
 32: u'001',
 33: u'001',
 34: u'001',
 35: u'001',
 36: u'001',
 37: u'001',
 38: u'001',
 39: u'001',
 40: u'001',
 41: u'001',
 42: u'001',
 43: u'001',
 44: u'001',
 45: u'001',
 46: u'001',
 47: u'001',
 48: u'001',
 49: u'001',
 50: u'001',
 51: u'001',
 52: u'001',
 53: u'001',
 54: u'001',
 55: u'001',
 56: u'001',
 57: u'001',
 58: u'001',
 59: u'001',
 60: u'001',
 61: u'001',
 62: u'001',
 63: u'001',
 64: u'001',
 65: u'001',
 66: u'001',
 67: u'001',
 68: u'001',
 69: u'001',
 70: u'001',
 71: u'001',
 72: u'001',
 73: u'001',
 74: u'001',
 75: u'001',
 76: u'001',
 77: u'00

In [5]:
 tradContes=set(nContes).intersection(set(conteLines))

## Basic information

In [6]:
print "mots-occurrences",lxaCorpus.number_of_word_tokens()
print "mots-types",lxaCorpus.number_of_word_types()

mots-occurrences 206136
mots-types 8720


## Séquences fréquentes
Liste des séquences de mots au dessus du *seuil* de fréquence pour :
- les mots seuls
- les bigrammes
- les trigrammes

In [7]:
def spreadListe(liste,cols):
    result=[]
    if isinstance(liste[0][0],tuple):
        locListe=[" ".join(seq)+" : "+unicode(nb) for seq,nb in liste]
    else:
        locListe=[seq+" : "+unicode(nb) for seq,nb in liste]
#     print locListe
    pos=[None for c in range(cols)]
    for n,l in enumerate(locListe):
        i=n%cols
        pos[i]=l
        if i+1==cols:
            result.append(pos)
            pos=[None for c in range(cols)]
    print (tabulate.tabulate(result))
    return 

In [8]:
seuilUnigram=1000
dUnigram=lxaCorpus.word_unigram_counter()
pUnigram=sorted(dUnigram.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print "mots de fréquence supérieure à %d"%seuilUnigram
freqUnigrams=[]
for w,f in pUnigram:
    if f>seuilUnigram:
#         print w,f
        freqUnigrams.append((w,f))
    else:
        break
spreadListe(freqUnigrams,6)

mots de fréquence supérieure à 1000
----------  ------------  ----------  ------------  -----------  ----------
ed : 10267  ñax : 5018    o : 4114    zix : 4051    zih : 4028   ah : 3784
ix : 3553   hoy : 3434    wta : 3101  tid : 2670    ossi : 2623  raq : 2604
zi : 2588   ñux : 2575    vta : 2554  wadzi : 2396  ziq : 2331   za : 2210
ap : 2035   cix : 1941    zu : 1935   ciz : 1836    wod : 1763   iq : 1738
peq : 1590  ñarzi : 1466  ñac : 1439  sec : 1392    xu : 1374    ih : 1320
----------  ------------  ----------  ------------  -----------  ----------


In [9]:
seuilBigram=200
dBigram=lxaCorpus.word_bigram_counter()
pBigram=sorted(dBigram.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print "séquences de deux mots de fréquence supérieure à %d"%seuilBigram
freqBigram=[]
for w,f in pBigram:
    if f>seuilBigram:
#         print " ".join(w),f
        freqBigram.append((w,f))
    else:
        break
spreadListe(freqBigram,4)

séquences de deux mots de fréquence supérieure à 200
---------------  --------------  -------------  ---------------
hoy ah : 525     ed wadzi : 516  za ñax : 494   ed za : 469
peq ah : 454     zih iq : 430    zih raq : 399  wadzi ñax : 398
ziq raq : 392    ah ñax : 368    oxz o : 358    zix dacyi : 299
ñax ñarzi : 291  o ix : 258      za zi : 252    ed wta : 250
tid ñaq : 249    ap whiz : 243   za ñux : 233   ed dos : 227
ed zi : 223      ñix hoy : 222   ev ed : 214    hoy ix : 212
---------------  --------------  -------------  ---------------


In [10]:
seuilTrigram=50
dTrigram=lxaCorpus.word_trigram_counter()
pTrigram=sorted(dTrigram.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print "séquences de trois mots de fréquence supérieure à %d"%seuilTrigram
freqTrigram=[]
for w,f in pTrigram:
    if f>seuilTrigram:
#         print " ".join(w),f
        freqTrigram.append((w,f))
    else:
        break
spreadListe(freqTrigram,4)

séquences de trois mots de fréquence supérieure à 50
-----------------  --------------------  -----------------  --------------------
ed za ñax : 151    ziq raq ixdaxd : 118  za ñax sec : 109   raq ixdaxd ix : 107
wod hoy ah : 94    riz wozix ap : 85     ed za ñux : 84     ah zih raq : 79
uz o wserix : 71   za ñax ñarzi : 68     yer hoy ah : 66    o ix paqh : 65
ed za zi : 62      za zi ñarzi : 59      o wserix ed : 58   ah ñax wsuyyi : 58
ed wahhi wod : 56  ed wadzi hoy : 56     za ñax wta : 55    wadzi ñax hoy : 55
za zi sec : 54     za ñax raq : 53       ykwh hoy ah : 53   wadzi zix dacyi : 53
ed hoy wozwh : 53  cix za ñax : 53       tid wsay xes : 51  ed wadzi ah : 51
-----------------  --------------------  -----------------  --------------------


# Découpage

In [53]:
dictBrokenWords=lxaCorpus.broken_words_left_to_right()
dictGlossWords={k:"-".join(v) for k,v in dictBrokenWords.iteritems()}
dictHyphenWords={k:"-".join(v) for k,v in dictBrokenWords.iteritems()}

In [104]:
dictFoundGloss={
    "zesheq":"docteur[nom1]",
    "zeshiq":"docteur[nom2]",
    "zesheqix":"docteur-DEF",
    "zad":u"jour",
    "teti":u"œil",
    "ixteti":u"1-œil",
    "heteti":u"2-œil",
    "hqiteti":u"3-œil",
    u"ñocyix":"ciel-DEF",
    "taxzix":"esprit-DEF",
    "wyaxdi":"serpent",
    "pyawsix":"bouteille-DEF",
    "ayrozixzi":"universel[adj]",
    "jexzix":"paysan-DEF",
    "jexzi":"paysan",
    "vixdi":"seigneur?",
    "vixdixi":"seigneur?",
    "wtex":"fils",
    "wtexxiq":"fils-PL",
    "ytexxi-h":"fils?",
    "zahhiq":"fille",
    "vodix":"jeune.fille-DEF",
    "jqtezqi":u"frère",
    "jqeqyoy":u"frérot",
    "wtewhiqyoy":u"sœurette",
    "zqixdi":u"garçon",
    "paq":u"père",
    "jaq":u"enfant",
    "jaqx":u"enfant-?",
    "caxz":"homme",
    "caxzix":"homme-DEF",
    "caxzixw":"homme-DEF-POSS",
    "sexi":"femme",
    "sexix":"femme-DEF",
    "pudy":"oiseau",
    "pudyix":"oiseau-DEF",
    "udyix":"chouette-DEF",
    "qarxi":"corbeau-PL?",
    "pqte":"grenouille",
    "pqteix":"grenouille-DEF",
    "dtiwwixi":"oie-DEF-PL",
    "sah":"chat",
    "sahhix":"chat-DEF",
    "cuw":"souris",    
    "cuwix":"souris-DEF",
    "diz":u"chèvre",
    "dizix":u"chèvre-DEF",
    "dizisoz":u"chevreau-PL",
    "dizisozzixi":u"chèvre-PETIT-DEF-PL",
    "uyrix":u"loup-DEF",
    "jtteqxix":u"ours-DEF",
    "tiwiy":u"âne",
    "qtirix":u"renard-DEF",
    "yuwix":u"pou-DEF",
    "yevvix":u"puce-DEF",
    "wser":u"forêt",
    "wserix":u"forêt-DEF",
    "sudyi":u"balle",
    "sudyix":u"balle-DEF",
    "sqexi":u"couronne",
    "wuyhax":"sultan",
    "sexdi":"roi",
    "sexdix":"roi-DEF",
    "zttiriyix":"diable-DEF",
    "zttiriyixw":"diable-DEF-GEN",
    "duyz":"or",
    "cizowhiqvteywi":"saucisse",
    "cizowhiqvteywix":"saucisse",
    "zahhiqix":u"château-DEF",
    "qtezñtihhi":"rouge+chaperon",
    "wyaxdijyazi":"serpent+feuille",
    "pqtesexdix":"grenouille+roi-DEF",
    "sexdizahhiqix":u"roi+château-DEF",
    "sexdiwtexxixw":"roi+fils?",
    "wtexxiwtexxix":"fils+fils?",
    "dyawsowhix":u"verre+cercueil-DEF",
    "duyzsudyi":"or+balle",
    "duyzsqexi":"or+couronne",
    "piqzoxaxz":"Fernand",
    "awsiveh":u"Cendrillon",
    "iywi":u"Élise",
    "qavuxgiy":u"Raiponce",
    "teqoxzi":u"Jorinde",
    "teqoxdiy":u"Joringel",
    u"wxiñrozi":u"Blanche-Neige",
    "sowhiq":"cercueil",
    "jqtexzix":"fontaine-DEF",
    "ctaxix":"lune",
    "wvoyyicaxz":"violon",
    "wixdi":"lit",
    "jtidiq":"gobelet",
    "yadixiq":"drap-PL",
    "cuxz":"bouche",
    "cuxzix":"bouche-DEF",
    "wvqed":"langue",
    "wsqtizziq":"tailleur",
    "wsqtizziqix":"tailleur-DEF",
    "powsiqix":u"pêcheur-DEF",
    u"jqtixziñuddiq":u"bûcheron(x+y?)",
    u"jqtixziñuddiqix":u"bûcheron(x+y?)-DEF",
    "ciwhiq":"patron",
    "soqsidtaqzix":u"cimetière-DEF",
    "soqsi":u"église",
    "soqsix":u"église",
    "wteccih":"clou-DEF",
    "ptiq":"plume-PL",
    "jtexxix":u"haricot-DEF",
    "dxowhix":u"braise-DEF",
    "ptiyzi":u"piège",
    "daxd":"temps",
    "taq":"an",
    "ctiys":"lait",
    "xowwiqxi":"lutin-PL",
    "raxz":"eau",
    "zixdaxd":"temps[DEF.PL]",
    "ixdaxd":"temps[INDEF?]",
    u"ñtixziq":"main",
    u"ñrozi":"blanc",
    u"jyta":"bleu?",
    "syed":"sage[SG]",
    "syedi":"sage-i[PL]",
    "syedh":"sage-h",
    "syedix":"sage-DEF",
    "yoyyi":"petit",
    "wcta":"petit?",
    "uxziqyod":"merveilleux",
    "uxziqyodi":"merveilleux-i",
    "uxziqyodh":"merveilleux-h",
    "rozuxziqyod":"merveilleux",
    "rozuxziqyodi":"merveilleux-i",
    "ykssiyod":"ensemble",
    "ykssiyodh":"ensemble",
    "ykssiyodi":"ensemble",
    "uhasxicyodi":"ingrat",
    "ixiwhi":u"seul/unique?",    
    "whoyyi":u"calme/tranquille?",    
    "whoyyizi":u"calme/tranquille?",    
    "whoyyiw":u"calme/tranquille?",    
    "whoyyih":u"calme/tranquille?",   
    "hqe":u"loyal",
    "uhqe":u"dé-loyal",
    "wacci":u"bien?",
    "waccix":u"bien?",
    "ayhwacci":u"pas-bien?",
    "ayyiwaccix":u"?pas-bien?",
    "dezi":u"bon[??]",
    "wheq":u"grand[??]",
    "cteqs":u"sombre[??]",
    "dacyi":u"vieux[??]",
    "dacyiw":u"vieux[??]",
    "wtezi":u"doux[??]",
    "exzi":u"mauvais[??]",
    "pyoxs":u"intelligent/avisé[SG]",
    "pyoxsi":u"intelligent/avisé[PL?]",
    "pahhodi":u"pauvre",
    "zityod":u"beau[??]",
    "payziqa":u"tombé[??]",
    "payzi":u"tombé[??]",
    "payziq":u"tombé[??]",
    "payzih":u"tombé[??]",
    "payzh":u"tombé[??]",
    "wvuqdhi":u"demander/répondre?-hi[PST]",
    "wvowi":"dormir",
    "hewwih":"dire???-hi[PST]",
    "sayzh":"s'appeler?-h",
    "rawsi":"se laver[FUT]",
    "yirizi":"vivre-zi[PST.3PL?]",
    "jyirih":"vivre-ih[PST.3PL?]",
    "zqossi":"boire[FUT]",
    "ossi":"pouvoir",
    u"ñreq":u"PRO.3SG-être[PRS]",
    u"ñric":u"PRO.3SG-être[PST?]",
    u"ñixzi":u"être[PST?]",
    "iq":u"être-COP[PRS.2SG]",
    u"raq":u"être-COP[PST]",
    u"ñraz":u"PRO.3SG-être/avoir[?]",
    u"ñarzi":"PRO.3SG-avoir[PST?]",
    u"wadzi":"dire[PST?]",
    u"ztezi":"mourir[PST]",
    u"ztezix":"mort-DEF",
    u"ztezixw":"mort-DEF-GEN",
    u"ñax":"PRO.3SG",
    u"ñaxw":"PRO.3SG-POSS",
    "cax":"PRO.1SG",
    "zu":"PRO.2SG",
    "zi":"PRO.3PL?/DEF.PL",
    "ziq":"PRO.3SG",
    "ziqiw":"PRO.3SG-POSS",
    "dqihi":u"PRO?/Gretel?",        
    u"ñrow":"DONT/POSS?",
    u"cox":"POSS.1SG",
    u"coxi":"POSS.1SG-PL",
    u"zox":"POSS.2SG",
    u"zoh":"POSS.2SG",
    u"wox":"POSS.3SG",
    u"woxi":"POSS.3SG-PL",
    "zix":"DEF[SG.?]",
    "zih":"DEF[SG.?]",
    "ed":"ET",
    "edwta":"ET-AUSSI",
    "ix":"1-INDEF",
    "ih":"INDEF",
    "he":"2",
    "hqi":"3",
    "tiyzwhi":u"premier",
    "kxdwhi":u"second",
    "hqizoi":u"troisième",
    "wkr":"7",
    "wisw":"6?",
    "heyr":"12",
    "zadi":u"TRÈS",
    "o":"DANS",
    "wec":"SUB",
    "cix":"MAIS",
    "ciz":"AVEC",
    "uzix":"SANS",
    "pqa":"DE(GEN)",
    "dos":"DE(PROV)",
    "wta":"COMME/AUSSI",
    "wyih":"NEG",    
#    "ah":"CONJ.QUE",
    }

In [105]:
for k,v in dictFoundGloss.iteritems():
    dictGlossWords[k]=k+"@"+v

In [106]:
texteHyphen=[]
texteGloss=[]
for nLineOrig,lineOrig in enumerate(texteBrut):
    lineHyphen=[]
    lineGloss=[]
    print nLineOrig+1
    for word in lineOrig.split(" "):
        if word:
            lineHyphen.append(dictHyphenWords[word])
            lineGloss.append(dictGlossWords[word])
    texteHyphen.append(" ".join(lineHyphen))
#    if nLineOrig+1==108:
#        print lineGloss
    texteGloss.append(" ".join(lineGloss))
#    print texteHyphen[-1]
#    print
#    if texteHyphen[-1]!=texteGloss[-1]:
    print texteGloss[-1]
    print

1
001 pqtesexdix@grenouille+roi-DEF iyyiq tiqxñixqos

2
o@DANS dacyi@vieux[??] zadi@TRÈS zixdaxd@temps[DEF.PL] cax@PRO.1SG suxxi pta woxi@POSS.3SG-PL texwsiq evpkyzh yirizi@vivre-zi[PST.3PL?] ziq@PRO.3SG ix@1-INDEF sexdi@roi wec@SUB ñarzi@PRO.3SG-avoir[PST?] ix@1-INDEF zahhiq@fille ziq@PRO.3SG raq@être-COP[PST] wta@COMME/AUSSI zityod@beau[??] ah wiyr weyix ziq@PRO.3SG zed ñaq wih wta@COMME/AUSSI cidih stexh uxzqi-zi-w ñriq daxd@temps zix@DEF[SG.?] wsoxxizi vta ñixzi@être[PST?]

3
o@DANS xtiqñizix ap wyehhih yta ziq@PRO.3SG ix@1-INDEF wheq@grand[??] cteqs@sombre[??] wser@forêt ed@ET ziq@PRO.3SG oxzi raq@être-COP[PST] ziq@PRO.3SG ix@1-INDEF jqtexz uxziq ix@1-INDEF dacciy yoxz

4
xtaq zih@DEF[SG.?] raq@être-COP[PST] cidih raqch dos@DE(PROV) sexdizahhiqix@roi+château-DEF uz o@DANS wserix@forêt-DEF ed@ET wahhi wod riz raxzih ed@ET xtaq ñux sizi-zi wod wvoyyizi ñux jeyz ciz@AVEC ix@1-INDEF duyzsudyi@or+balle wec@SUB raq@être-COP[PST] ñixziw stiqiwhi yidihtet

5
ix@1-INDEF zad@jour wsihi ziq@


548
ziqap wsay zu@PRO.2SG yazi ix@1-INDEF duyzwciz peqa-qjitzi ix@1-INDEF ñiy ctixdzi saq ed@ET pudyi ed@ET ctiqsiyodi zkq

549
zih@DEF[SG.?] roy ñux xes wkxiw ec ed@ET wta@COMME/AUSSI hadiq ro ap whiz ciz@AVEC hoxdixi ed@ET vqteri-q req ykssi

550
sexdix@roi-DEF wixzhi xu juz hoy ayyi duyzwciz-i o@DANS yaxzih ed@ET zi@PRO.3PL?/DEF.PL ctahhi aqjitzi zad@jour ed@ET xah peq ah pta hoxdixi ptiqzodi

551
ziqvta jyir zih@DEF[SG.?] ayhwaccix ytiw-wi-h vta ih@INDEF wsoj ed@ET sexdix@roi-DEF ed@ET zix@DEF[SG.?] hqe@loyal teñaxxiw sytizhi wod uz wec@SUB stej-ctixz wta@COMME/AUSSI cax@PRO.1SG wyih@NEG ossi@pouvoir suxxi sixzi zic

552
wta@COMME/AUSSI wityizi zi@PRO.3PL?/DEF.PL ap whiz eriq ñarih hoy zi@PRO.3PL?/DEF.PL xtaizi zix@DEF[SG.?] jk ñreq@PRO.3SG-être[PRS] sexdizahhiqix@roi+château-DEF pqa@DE(GEN) zih@DEF[SG.?] dkyzxi had jeizi

553
zix@DEF[SG.?] hqe@loyal teñaxxiw wadzi@dire[PST?] ah sexdix@roi-DEF wsuyyi jyori xizi vta wsojih ed@ET rixhi vta ñac

554
ctawsi jqoxdiq tid sexdizahhiqix@r

zi@PRO.3PL?/DEF.PL sec wxaqh hoy ix@1-INDEF ed@ET jqeqyoy@frérot royyi ytiddi wod xiz ed@ET zqossi@boire[FUT] ap zih@DEF[SG.?] jyoxsi-xzi raxz@eau cix@MAIS wtewhiqyoy@sœurette suxxi ñteqi ñreqzax zix@DEF[SG.?] cucyizi zix@DEF[SG.?] ziq@PRO.3SG zqossiq ap cod jyoriq hoy ix@1-INDEF hodiq zix@DEF[SG.?] ziq@PRO.3SG zqossiq ap cod jyoriq hoy ix@1-INDEF hodiq

948
zu@PRO.2SG cta ossi@pouvoir zqossi@boire[FUT] xedih qtajhi ñux za wta@COMME/AUSSI jyoriq zu@PRO.2SG hoy ix@1-INDEF hodiq ed@ET wtex-ziq-q-oriq cod

949
wstexh ñax@PRO.3SG raq@être-COP[PST] cidih hteq-whod wadzi@dire[PST?] ñax@PRO.3SG wta@COMME/AUSSI rixhiq tid ciz@AVEC ah zqossi@boire[FUT] hoy ro xtiwhi daxd@temps secciq hoy xedih raxz@eau

950
cix@MAIS za zi@PRO.3PL?/DEF.PL sec hoy zix@DEF[SG.?] xtiwhi soyzi ñteqhi wtewhiqyoy@sœurette ah zix@DEF[SG.?] cucyizi zix@DEF[SG.?] ziq@PRO.3SG zqossiq ap cod jyoriq hoy ix@1-INDEF uyr zix@DEF[SG.?] ziq@PRO.3SG zqoxsiq ap cod jyoriq hoy ix@1-INDEF uyr

951
zu@PRO.2SG cta oxdixhoxd zqossi@boi

ñaxw@PRO.3SG-POSS whoyyizi@calme/tranquille? wod xu vta htiiqxi ed@ET jqtissizi ih@INDEF yoyyi@petit whkssi ap hadih peq ah vqteri ñreqzax zih@DEF[SG.?] wcadhi ed@ET dqihi@PRO?/Gretel? dar wod hoy ah dxari ytew vta quzix

1348
za ñteqhi zi@PRO.3PL?/DEF.PL oxzi pqa@DE(GEN) whuix ix@1-INDEF hkxz whicci ziq@PRO.3SG qtajhi

1349
ñric@PRO.3SG-être[PST?] iq@être-COP[PRS.2SG] zih@DEF[SG.?] ziquzi

1350
wec@SUB dxari-q vta cox@POSS.1SG quzi

1351
jteqxixi wraqizi

1352
zih@DEF[SG.?] iq@être-COP[PRS.2SG] jaqi wheqci-xw wuw

1353
ziq@PRO.3SG huzi-q ed@ET ñroxi-q dixxic tiqh ñuw

1354
ed@ET wvowhi roziqi uzix@SANS ah yazi wod peqwhkqqi

1355
ñaxw@PRO.3SG-POSS ziq@PRO.3SG wkxhiw ah hadih wcadhi zityodh jqtissizi ih@INDEF wheqh whkssi ap ed@ET dqihi@PRO?/Gretel? whtezhi ix@1-INDEF ñiy yoyyi@petit quzi oxz ed@ET dteqzi wod qodhod hoy dezi@bon[??] ziqciz

1356
vyuzwiyod dos@DE(PROV) zteqix ev ed@ET ix@1-INDEF tiyzdacciy sexi@femme ziq@PRO.3SG whtehhizi wod vta ix@1-INDEF sqkssi sec uz

1357
ñaxw@PRO.

1747
zih@DEF[SG.?] wsay qodhod wcadi cod wadzi@dire[PST?] wsqtizziqix@tailleur-DEF hed jqtezih uz ap wsajih wsaq wod ih@INDEF eqzixhyodh whkssi ed@ET wcuqhi ñexxoxd vta

1748
zih@DEF[SG.?] iq@être-COP[PRS.2SG] zed riy jizwh ah tid pteqwh jyoriq ptiqzod ciz@AVEC coh wkhtet htixshi ñax@PRO.3SG wahhi jqtezih riz wozix aq wod ed@ET wkizi roziqi cix@MAIS ñax@PRO.3SG raq@être-COP[PST] wta@COMME/AUSSI dyaz ah ñax@PRO.3SG whazod hed whteqqi ed@ET whteqqi whoxd

1749
yudhix ap zix@DEF[SG.?] wtezi@doux[??] ñexxoxd hqti-xdhi ocozyiqhoz ñix hoy rtiddix ñreq@PRO.3SG-être[PRS] ziq@PRO.3SG waz puyzh ap pyuiq ed@ET zi@PRO.3PL?/DEF.PL sec xu pykrixzi ed@ET wahhi wod vta jqtezih

1750
ñric@PRO.3SG-être[PST?] ñaq oxzjuzh tiq

1751
wadzi@dire[PST?] wsqtizziqix@tailleur-DEF ed@ET ted zic rtis

1752
pyuiqxi peqwhez ossi@pouvoir zaxws cix@MAIS sec ih@INDEF tetijyos iphiq hoyjadi

1753
za hajhi wsqtizziqix@tailleur-DEF htaycezodñizix hed ix@1-INDEF whuc-v htet ed@ET wctissizi iphiq zic

1754
ossi@pouvoir coxz

...


11037
zucci xkwdiqqod-i dtaw wadzi@dire[PST?] zrtiqdix tid royyi syte-ri hqtiih ed@ET ñuddi xedih wctah jqtixzi hoy stessixih

11038
riz whtiqs oyz jqtixz-iq reqiw wcuyi caz whqasw

11039
ro jiñteriq ossi@pouvoir wta@COMME/AUSSI cidih wec@SUB o@DANS wheqi dqta-zod-i cixxiwsiq

11040
tid ñarzi@PRO.3SG-avoir[PST?] ayyiqizi ptaih soyix oxz ed@ET hqeizi zix@DEF[SG.?] ñiyyodi dqar raq@être-COP[PST] riy peqr-aqih cix@MAIS zih@DEF[SG.?] peqjowhqizi hqti raq@être-COP[PST] wta@COMME/AUSSI dyah ah zix@DEF[SG.?] wvqaxd uz odix ed@ET qirxix yussizi wod wta@COMME/AUSSI ñuqhodh waccix@bien? ah coh wcussi ñrozi@blanc wstid sec o@DANS syic-ci ed@ET xu sax tid ossi@pouvoir pta zih@DEF[SG.?] uz odix

11041
wi ñreq@PRO.3SG-être[PRS] o@DANS whtaq ed@ET dqoxiq ciz@AVEC tiqiw zucci ctiysijqtezwpttiw

11042
ñreq@PRO.3SG-être[PRS] o@DANS iq@être-COP[PRS.2SG] rticciyodi

11043
xu wsay tid sayzi vta xedyi peys wadzi@dire[PST?] qewixqtez

11044
zih@DEF[SG.?] raq@être-COP[PST] edwta@ET-AUSSI ih@INDEF vtapuxz zox

ec xahhix sec qteriqxi ñtic ed@ET wahhi wod hoy ah zqossi@boire[FUT] zih@DEF[SG.?] ixi dyaw rox iphiq zih@DEF[SG.?] axzih ed@ET jidkxzhi hoy wozwh ah xossi ciz@AVEC ñerizih

11389
ñaxw@PRO.3SG-POSS dos@DE(PROV) xu ciz@AVEC sxovyi-x o@DANS ñtaxzix ñix hoy axpteqiqix ed@ET wvuqdhi@demander/répondre?-hi[PST] ec ñax@PRO.3SG royyi wodi ñac ñric@PRO.3SG-être[PST?] ziq@PRO.3SG raq@être-COP[PST] ñaxw@PRO.3SG-POSS paq@père

11390
qteriqix dar ñac odix ix@1-INDEF sxayz-ixzi teqipodix wta@COMME/AUSSI ñax@PRO.3SG payzh@tombé[??] ec cix@MAIS zih@DEF[SG.?] raqizi ossi@pouvoir ytixdi pteq ñax@PRO.3SG sec vta jixixi odix ed@ET wta@COMME/AUSSI wyed ñax@PRO.3SG ytew vta qteriqxi hoy zi@PRO.3PL?/DEF.PL ossi@pouvoir suxxi qteqi ñriqsix aqci iyyiq jix

11391
ceziqix whez ñixxi o@DANS ix@1-INDEF sqed ed@ET wta@COMME/AUSSI ciz@AVEC dyaz pequxzqoxd ñreq@PRO.3SG-être[PRS] whtiqs ed@ET havvi-q ñixziw wtex@fils raq@être-COP[PST]

11392
za qteriqxi ñarzi@PRO.3SG-avoir[PST?] ptaih ñraz@PRO.3SG-être/avoir[?] zi@PRO

11741
hoyptiyzodrow raq@être-COP[PST] ix@1-INDEF ap zi@PRO.3PL?/DEF.PL wheqi udyiq pqa@DE(GEN) wserix@forêt-DEF wec@SUB ñizziq ñeqxudyiq seccih oxz o@DANS ix@1-INDEF yazi ed@ET za zadix jqtez pqic huqzi zix@DEF[SG.?] ossi@pouvoir reri wod uz ap pqkdh peq zi@PRO.3PL?/DEF.PL axzqi pudyi wec@SUB wsqodi-q wta@COMME/AUSSI daxwsi peqptiqziy-odh xtaq zi@PRO.3PL?/DEF.PL wiq zix@DEF[SG.?]

11742
za saqyix ec ceqdixix sec uz o@DANS yazix peq ah ñixhi ñte jyir ñax@PRO.3SG wta@COMME/AUSSI peqwsqtissih riz ah wi udyix@chouette-DEF wec@SUB waz o@DANS ix@1-INDEF sqed ah ñax@PRO.3SG whkqhizi oxz ed@ET peqhayhi wox@POSS.3SG ñiqqi ah ziq@PRO.3SG uzi o@DANS yazix waz zih@DEF[SG.?] peqpt-i-qziyodwhi uñkqi ñax@PRO.3SG xedixwoxzi ñarzi@PRO.3SG-avoir[PST?] wih

11743
zih@DEF[SG.?] quyyizi ciz@AVEC tetxixi ed@ET wta@COMME/AUSSI uz wec@SUB ec zih@DEF[SG.?] uzix@SANS roziqi suxxi paqi ytew vta ix@1-INDEF ed@ET wyudi ix@1-INDEF

11744
tañ tid sixziq zod xes wadzi@dire[PST?] caxzix@homme-DEF zu@PRO.2SG iq@être-CO

12141
riz zu@PRO.2SG ñraz@PRO.3SG-être/avoir[?] syessix iq@être-COP[PRS.2SG]

12142
wvuqdhi@demander/répondre?-hi[PST] zix@DEF[SG.?] dacyi@vieux[??]

12143
zih@DEF[SG.?] iq@être-COP[PRS.2SG] ossi@pouvoir cozxah ixzxu wraqizi vodix@jeune.fille-DEF cix@MAIS zix@DEF[SG.?] iq@être-COP[PRS.2SG] eriq iyyiri

12144
htixsiq zu@PRO.2SG wyih@NEG ossi@pouvoir vta ah o@DANS zad@jour iq@être-COP[PRS.2SG] zih@DEF[SG.?] hqi@3 taq@an wozix zu@PRO.2SG sec wvuqdhi@demander/répondre?-hi[PST] zix@DEF[SG.?] dacyi@vieux[??] cix@MAIS xu iq@être-COP[PRS.2SG] zih@DEF[SG.?] peqjo xu sax ro ossi@pouvoir ytixdiqi jyori waccix@bien?

12145
vodix@jeune.fille-DEF jyir peqwsqtissih

12146
roy o@DANS xu peqwhtez-i cod wadzi@dire[PST?] ñux tid ñaq ñriqsix wytidh iyyiq rixxiq wec@SUB tid sax jizi ec ñttiyv

12147
tid ñaq dteqh ayh ñraz@PRO.3SG-être/avoir[?] o@DANS ñaq wadh ed@ET o@DANS ñaq ayhoz rtiqih hoypqizw ciz@AVEC cod

12148
wixz cod zed ossi@pouvoir jeqh

12149
zix@DEF[SG.?] dacyi@vieux[??] royyi ossi@pouvoir wod

ziq@PRO.3SG caxdyiq ih@INDEF wtec o@DANS wseix vta zih@DEF[SG.?] rixwhqi jadjix wadzi@dire[PST?] ñax@PRO.3SG

12438
ta wta@COMME/AUSSI cta zix@DEF[SG.?] caxdyi wadzi@dire[PST?] stejcaxzix wseix ñeyziq riy xes zi@PRO.3PL?/DEF.PL wisw@6? hociq tid ñaq hoyjadi

12439
tid ñaq hqar-yh

12440
ec iphiqcozzad-ix whid ñax@PRO.3SG odix ap ñiwh-ix ed@ET zix@DEF[SG.?] jyir pteqh oxz peq ah pta xedih ah wvowi@dormir

12441
yozh iphiq sec saqyix oxz o@DANS whuix ed@ET wadzi@dire[PST?] ziq@PRO.3SG caxdyiq ix@1-INDEF ñiwhi-wse vta zih@DEF[SG.?] rixwhqi jadjix

12442
wsay tid jqoxdi zkqih ñix hoy wciz-ix

12443
xit zih@DEF[SG.?] ptaq rtiqi wraqizi stejcaxzix zi@PRO.3PL?/DEF.PL vaq hociq ziq@PRO.3SG ixzxu iq@être-COP[PRS.2SG] hoyjadi ñeyziq ñiwh-ix xes uz

12444
tid ñaq hqar-yh

12445
ñax@PRO.3SG qiz ap whiz cix@MAIS yozh iphiq jidkxzhi ñiwh-ix ah ñoxsi

12446
zih@DEF[SG.?] raqizi xu ossi@pouvoir ytixdi pteq zix@DEF[SG.?] wxujyizi ed@ET payzh@tombé[??] ed@ET jqtissizi jixih

12447
stejcaxzix ctahhi yazi

## Concordances

### Texte brut

In [16]:
def displayConcordance(concordance,fenetre=50):
    blancGauche=concordance.startswith(" ")
    blancDroite=concordance.endswith(" ")

    dfConcordance=pd.DataFrame(columns=["conte","num","gauche",concordance,"droite"])
    p=re.compile("(.*)(%s)(.*)"%concordance,re.U)
    for nLine,line in enumerate(texteBrut[:]):
        occurrences=re.findall(p,line)
        if occurrences:
            for occurrence in occurrences:
                if len(occurrence[0])>fenetre:
                    gauche=occurrence[0][-fenetre:]
                else:
                    gauche=occurrence[0]
                if not blancGauche and gauche and gauche[-1]!=" ":
                    gauche=gauche+"-"

                if len(occurrence[2])>fenetre:
                    droite=occurrence[2][:fenetre]
                else:
                    droite=occurrence[2]
                if not blancDroite and droite and droite[0]!=" ":
                    droite="-"+droite

                dfLine={"conte":lineConte[nLine],"num":nLine+1,"gauche":gauche,concordance:occurrence[1],"droite":droite}
                dfConcordance=dfConcordance.append(dfLine,ignore_index=True)

    # dfStyler=dfConcordance.style.set_properties(subset=[concordance],**{'font-weight': 'bold'})
    dfStyler=dfConcordance.style.set_properties(subset=['gauche'],**{'text-align': 'right'})
    dfStyler=dfConcordance.style.set_properties(subset=['droite'],**{'text-align': 'left'}).hide_index()
    display(dfStyler)
    return


In [17]:
displayConcordance(" whoyyi")

conte,num,gauche,whoyyi,droite
001,69,hizi xu sexdiwtexxix ed vqoxwiwwix oxz o redxix ed,whoyyi,-zi wod jadriz zic ed ñaxw ñtiqhi wruycizi ap dytiz
002,84,shi wod ytixdi ec ed ixziyod wadzi zix ro sax ossi,whoyyi,zix xedih wosqiqi whiz ixz o soqsix ziq iq ziq ox
003,163,ñux har wta,whoyyi,cix xkwdiqqodñizix yez ñixzi ossi ñari pqiz
003,216,rzi tizh zih ed coxowhqixi peqyaxdhi ah ñux wsuyyi,whoyyi,-w peq ix zecwhey
003,225,ñux jyir,whoyyi,-h peq ix zecwhey ed za ñux ossi suxxi peqwraqi wod
004,234,x kxdwhi raq wta zuc ah ziq wyih ossi raq xedih ah,whoyyi,ev ciz ñac
004,248,c ed wadzi ah ñax wyih ossi rozwhi ñraz ñax wsuyyi,whoyyi,ev ciz wox kxdwhi wtex ñax raq wta zuc wta zih os
004,263,xshi zidxix ed dar ossi ix ykz pqa wod ed whez wta,whoyyi,wec ec ñax raq ap whix
004,324,ho zed,whoyyi,wadzi ñaxw sexi ziq iq ayyiqizi wta caxdi ziq ñaq
004,337,hixix dos zqixdix ziqoxz htixzhi wod ih syaqh jtay,whoyyi,-zi ñteryijtixsix ciz sxorix riz wozix ap ed wahhi


In [18]:
displayConcordance(" wvtiqqi")

conte,num,gauche,wvtiqqi,droite
011,988,x za qtazkqih wadzi tid zteq ap weqd ñrow tid wsay,wvtiqqi,-w oxzi ñiq suxxi ñux ossi axzih ixz ciz huxdh ñtiq
019,1716,ñax hqeizi ñax ñarzi ñteqh dayh ed,wvtiqqi,-zi tetxixi rozh ev ed wadzi ñraz iq zih zed zu wod
057,4693,ax ptiqxi zih jtiqd ziq yoddiq uzixpeq coh wyeh ed,wvtiqqi,-q uzwodhix
060,5163,ap ritix peq zix cix za zix sec hoy wsoyzradhiqxi,wvtiqqi,-zi zi ritix peq zix ciz ziqiw jtewwiq ed royyi oss
060,5173,wsoyzradhiqxi royyi,wvtiqqi,ritix peq zix cix zix uzwhtezhi jyeh ih jqtey wta
071,5913,hui ñreq duyrih ed zteqix raq ap tiqx ed roxzuiqxi,wvtiqqi,-h ciz tiqxwhtixdiq
166,11395,zi dos xu uz ap ñuyix ed zih sax xes rtiqi ah ñaxw,wvtiqqi,-zi tetxixi ev za ñax pqa cteqsih sec uz o zih syaq
166,11459,i ed za ñux ossi royyi dophi wod ciz ñac ñarzi ñax,wvtiqqi,-h ñixzi oxzi o syovvix ed wah zix exzwsajwpuyzi zr
169,11625,zi iq,wvtiqqi,-h oxzi o stiyziqix wraqizi ñax o ceqdix wsay zi pt
172,11710,wh azwsoyhi zic ziq diqxi royyi rtiqi waccix iyyiq,wvtiqqi,-zi ritix peq zic


In [19]:
conte="031"
if conte in tradContes:
    print "\n".join(conteLines[conte])

9471 031 la jeune fille sans mains
9472 il était une fois il y a quelques jours à l époque où la farine des villageois était écrasée à la meule de pierre un meunier qui avait connu des temps difficiles
9473 il ne lui restait plus que cette grosse meule de pierre dans une remise et derrière un superbe pommier en fleur
9474 un jour tandis qu il allait dans la forêt couper du bois mort avec sa hache au tranchant d argent un curieux vieillard surgit de derrière un arbre
9475 a quoi bon te fatiguer à fendre du bois
9476 dit il
9477 ecoute si tu me donnes ce qu il y a derrière ton moulin je te ferai riche
9478 qu y a t il derrière mon moulin sinon mon pommier en fleurs
9479 pensa le meunier
9480 il accepta donc le marché du vieil homme
9481 dans trois ans je viendrai chercher mon bien gloussa l étranger avant de disparaître en boitant derrière les arbres
9482 sur le sentier en revenant le meunier vit son épouse qui volait à sa rencontre les cheveux défaits le tablier en bataille
9483 mon épo

### Texte glosé

In [20]:
def displayGlossConcordance(concordance,fenetre=50):
    blancGauche=concordance.startswith(" ")
    blancDroite=concordance.endswith(" ")

    dfConcordance=pd.DataFrame(columns=["conte","num","gauche",concordance,"droite"])
    p=re.compile("(.*)(%s)(.*)"%concordance,re.U)
    for nLine,line in enumerate(texteGloss[:]):
        occurrences=re.findall(p,line)
        if occurrences:
            for occurrence in occurrences:
                if len(occurrence[0])>fenetre:
                    gauche=occurrence[0][-fenetre:]
                else:
                    gauche=occurrence[0]
                if not blancGauche and gauche and gauche[-1]!=" ":
                    gauche=gauche+"-"

                if len(occurrence[2])>fenetre:
                    droite=occurrence[2][:fenetre]
                else:
                    droite=occurrence[2]
                if not blancDroite and droite and droite[0]!=" ":
                    droite="-"+droite

                dfLine={"conte":lineConte[nLine],"num":nLine+1,"gauche":gauche,concordance:occurrence[1],"droite":droite}
                dfConcordance=dfConcordance.append(dfLine,ignore_index=True)

    # dfStyler=dfConcordance.style.set_properties(subset=[concordance],**{'font-weight': 'bold'})
    dfStyler=dfConcordance.style.set_properties(subset=['gauche'],**{'text-align': 'right'})
    dfStyler=dfConcordance.style.set_properties(subset=['droite'],**{'text-align': 'left'}).hide_index()
    display(dfStyler)
    return

In [108]:
displayGlossConcordance(u" viqy",50)

conte,num,gauche,viqy,droite
001,14,teau-DEF coxi@POSS.1SG-PL sytiziq coxi@POSS.1SG-PL,viqy,-iq ed@ET coxi@POSS.1SG-PL tizi-ywhixi ed@ET ñrow@D
015,1392,ziq@PRO.3SG whez wheqi sawwi-q puyzi ap,viqy,-iq ed@ET tizi-ywhixi
015,1410,dqihi@PRO?/Gretel? sawhizi,viqy,-i-qxi ed@ET tizi-ywhixi uz ap peqsytizi-h ed@ET ña
021,1901,wcussi sytiziq wadzi@dire[PST?] zix@DEF[SG.?] ixi,viqy,-iq ed@ET tizi-ywhixi jaz zix@DEF[SG.?] axzix
021,1904,PRO.3SG stejhi za edwta@ET-AUSSI xki sytiziq ed@ET,viqy,-iq ed@ET tizi-ywhixi hoy woxi@POSS.3SG-PL he@2 zte
021,1939,@or ed@ET wteyr-oxzroqsih steyi ed@ET ih@INDEF vaq,viqy,-ijqeziqizi wse xiz hoy ñixzi@être[PST?]
039,3353,-INDEF wixd ap weqh ojixhqti ciz@AVEC sxav-vi-q ap,viqy,-iq htivviqxi raq@être-COP[PST] duyzj-qeziqizi rudd
062,5442,whi wsuyyi cax@PRO.1SG wacyi huwoxz ap vqoxwiwwixw,viqy,-iq wec@SUB yta uzi uxziq cewwih o@DANS wserix@forê
062,5444,uvoir wheqh jizqi ñax@PRO.3SG paxzh sux heñuxzqizi,viqy,-iq ed@ET jyir edwta@ET-AUSSI hoy whix
062,5447,a@petit? zkq ñarzi@PRO.3SG-avoir[PST?] puxzih ayyi,viqy,-i-qxi ed@ET jtaqih zic waccix@bien? o@DANS ix@1-IN


In [260]:
displayGlossConcordance(u"^\d\d\d ",80)

conte,num,gauche,^\d\d\d,droite
001,1,,001,pqtesexdix@grenouille+roi-DEF iyyiq tiqxñixqos
002,80,,002,sah@chat ed@ET cuw@souris o@DANS jeptiyyiwsaj
003,140,,003,caqoijaqxih
004,232,,004,irixhkq-ih ec ix@1-INDEF ziq@PRO.3SG zqed uz peq ah ytiqi pqkdh ah sixzi
005,445,,005,uyrix@loup-DEF ed@ET zi@PRO.3PL?/DEF.PL wkr@7 dizisoz@chevreau-PL
006,505,,006,zix@DEF[SG.x] hqe@loyal teñaxxiw
007,654,,007,zix@DEF[SG.x] dezi@bon[??] ñaxziy
008,744,,008,zix@DEF[SG.x] uxziqyodi@merveilleux-i wvoyyicaxz@violon
009,795,,009,zi@PRO.3PL?/DEF.PL heyr@12 jqtezqi@frère
010,900,,010,vas


## Calculer une hiérarchie de signatures morphologiques
Les signatures morphologiques de Linguistica correspondent à un paradigme flexionnel réduit aux variantes attestées dans le corpus pour un même lexème. Hiérarchiser les signatures permet d'étudier les regroupements possibles entre paradigmes partiels.

In [15]:
morphGraph=nx.Graph()
sigGraph=nx.DiGraph()

In [16]:
for sig in lxaCorpus.signatures():
    for a,b in combinations(sig,2):
        if (a,b) not in morphGraph.edges:
            morphGraph.add_edge(a,b,weight=1)
        else:
            morphGraph[a][b]["weight"]+=1

### Cliques maximales parmi les signatures

In [17]:
maxCliques=list(nx.find_cliques(morphGraph))
# maxCliques

In [18]:
dCliques={tuple(c):set() for c in maxCliques}
for sig in lxaCorpus.signatures():
    sSig=set(sig)
    for clique in dCliques:
        sClique=set(clique)
        if sSig.intersection(sClique)==sSig:
            dCliques[clique].add(sig)
print "Les cliques maximales et leurs sous-cliques sont :"
print
for cMax,lCliques in dCliques.iteritems():
    if len(lCliques)>1:
        print cMax,":"
        print "\t",list(lCliques)
    else:
        print cMax
    print

Les cliques maximales et leurs sous-cliques sont :

(u'NULL', u'i', u'iw') :
	[(u'NULL', u'i'), (u'NULL', u'i', u'iw')]

(u'NULL', u'i', u'\xf1iz') :
	[(u'NULL', u'i'), (u'NULL', u'i', u'\xf1iz')]

(u'NULL', u'ci')

(u'NULL', u'i', u'ix', u'ixi', u'ixw') :
	[(u'NULL', u'i', u'ix', u'ixi'), (u'NULL', u'ix'), (u'NULL', u'i', u'ix'), (u'NULL', u'ix', u'ixw'), (u'NULL', u'i'), (u'NULL', u'i', u'ix', u'ixi', u'ixw'), (u'NULL', u'ixi')]

(u'hi', u'i', u'h') :
	[(u'h', u'i'), (u'h', u'hi', u'i')]

(u'NULL', u'q', u'ywi') :
	[(u'NULL', u'q'), (u'NULL', u'q', u'ywi')]

(u'NULL', u'qi', u'whi') :
	[(u'NULL', u'qi'), (u'NULL', u'qi', u'whi'), (u'NULL', u'whi')]

(u'NULL', u'zi', u'q', u'h', u'xzi', u'w') :
	[(u'xzi', u'zi'), (u'NULL', u'h', u'w'), (u'q', u'zi'), (u'NULL', u'q', u'w'), (u'NULL', u'h', u'q'), (u'q', u'xzi', u'zi'), (u'NULL', u'q', u'zi'), (u'NULL', u'q'), (u'NULL', u'h', u'zi'), (u'h', u'q'), (u'h', u'q', u'zi'), (u'NULL', u'q', u'w', u'xzi'), (u'NULL', u'w'), (u'NULL', u'zi'), (u'

In [ ]:
sigMax={}
for k,l in dCliques.iteritems():
    for e in l:
        if e not in sigMax:
            sigMax[e]=set()
        sigMax[e].add(k)
sigMax[(u'NULL', u'ix')]

In [24]:
lxaCorpus.stems()
lxaCorpus.affixes()
dSignatureStems=lxaCorpus.signatures_to_stems()

In [23]:
keySignatures=[]
for k in lxaCorpus.signatures_to_stems().keys():
    if len(k)>4:
        keySignatures.append(k)

In [29]:
for k in keySignatures:
    print ", ".join(k)
    print "\t",", ".join(lxaCorpus.signatures_to_stems()[k])
    print

NULL, h, q, w, zi
	whoyyi, pitqi, wvtiqqi, ñixhi, tadi, ñuddi

NULL, si, sih, siq, sizi
	ctiq, ztis, vyus, rtis, hqks

NULL, h, q, xzi, zi
	yowhi, whkqhi, vawwi, wrtecci, ñevvi, qkwhi, jaxsi, sqaryi

NULL, i, ih, iq, izi
	rixh, yteph, wraq, redh, texws, ñitw

NULL, i, ix, ixi, ixw
	wrixz, ñisw, zqixd, jtteqx, ñiwh

NULL, h, q, x, zi
	yazi, paxdi, vqteri, zaxwi, reswi, ñassi, syovvi, ñroyi, qidxi, rtadi



In [ ]:
lxaCorpus.signatures_to_words()
lxaCorpus.affixes_to_signatures()

In [ ]:
lxaCorpus.stems_to_signatures()["zesheq"]

In [ ]:
lxaCorpus.words_in_signatures()
lxaCorpus.words_to_signatures()
lxaCorpus.words_to_sigtransforms()

## Words manifolds and syntactic word neighborhood

In [ ]:
# sorted(lxaCorpus.words_to_neighbors().keys())
lxaCorpus.words_to_neighbors()["jexzix"]

In [ ]:
network=lxaCorpus.neighbor_graph()
# plt.figure(1,figsize=(100,100))
# nx.draw(network)
# plt.show()

In [ ]:
lxaCorpus.words_to_contexts()

## Phonology

In [ ]:
lxaCorpus.phone_unigram_counter()
lxaCorpus.phone_bigram_counter()
lxaCorpus.phone_trigram_counter()

## Voisinages

In [22]:
print (", ".join(lxaCorpus.words_to_neighbors()["wvuqdhi"]))

htixshi, qtajhi, jyir, ctahhi, hed, roy, ñaq, royyi, ñteqhi
